In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
import base64
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps


from ProjectOne import AnimalShelter





###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "userpassword"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())



app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height = 150)),
    html.Center(html.B(html.H1('SNHU CS-340 Angelica Strack Dashboard'))),
    html.Hr(),
    
    #Button information  - uncomment this and the button callback to use buttons instead of radios
    #html.Div(className='row', 
    #    style={'display' : 'flex'}, 
    #       children=[
    #           html.Button(id='submit-button-one', n_clicks=0, children = 'Water Rescue'),
    #             html.Button(id='submit-button-two', n_clicks=0, children = 'Mountain or Wilderness'), 
    #            html.Button(id='submit-button-three', n_clicks=0, children = 'Disaster or Individual Tracking')
    #        ]),
    
    #radio selection is the best option at the moment because the logic works better and requires less work for the user
    dcc.RadioItems(
        id = 'radioItem',
        options=[
            {'label' : 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain or Wilderness', 'value' : 'MoW'},
            {'label': 'Disaster or Individual Tracking', 'value' : "DoIT"},
            {'label' : 'All Items', 'value' : 'all'}
        ],
        value = 'all',
        labelStyle = {'display' : 'inline-block'}
    ),
    
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
               
        data = df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native", 
        sort_mode="multi", 
        column_selectable=False, 
        row_selectable=False,
        row_deletable=False,
        selected_columns=[], 
        selected_rows=[],
        page_action="native", 
        page_current=0,
        page_size=10,
        
    ),
    html.Br(), 
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(
    Output('datatable-id', "data"),
    [Input('radioItem', 'value')])

def update_output(value):

    if value == 'all':
        df = pd.DataFrame.from_records(shelter.read({}))
    elif value == 'WR':
        df = pd.DataFrame.from_records(shelter.read({"age_upon_outcome_in_weeks" : {"$gte":26, "$lte":156}, 
            "sex_upon_outcome" : "Intact Female", 
            "breed" : {"$in" : ["NewFoundland", "Labrador Retriever Mix", "Chesapeake Bay Retriever"]}}))
    elif value == 'MoW':
        df = pd.DataFrame.from_records(shelter.read({"age_upon_outcome_in_weeks" : {"$gte":26, "$lte":156},
            "sex_upon_outcome" : "Intact Male",
            "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                                " Siberian Husky", "Rottweiler"]}}))
    elif value == 'DoIT':
        df = pd.DataFrame.from_records(shelter.read({"age_upon_outcome_in_weeks" : {"$gte":20, "$lte":300},
            "sex_upon_outcome" : "Intact Male", 
            "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                                " Bloodhound", "Rottweiler"]}}))   
    
    return df.to_dict('records')
        
#Callback for using buttons as the input - the button logic is broken and needs work
#@app.callback(
#   Output('datatable-id', "data"), [Input('submit-button-one', 'n_clicks'),
#            Input('submit-button-two', 'n_clicks'), Input('submit-button-three', 'n_clicks')])    

#Button logic - broken and needs work. This logic forces multiple button presses to move between filtering
#def on_click(bt1, bt2, bt3):
#    if (int(bt1) == 0 and int(bt2)==0 and int(bt3)==0):
#        df = pd.DataFrame.from_records(shelter.read({}))
#    elif (int(bt1) > int(bt2) and int(bt1) > int(bt3)):
#        df = pd.DataFrame(list(shelter.read({"age_upon_outcome_in_weeks" : {"$gt":26, "$lt":156}, 
#                "sex_upon_outcome" : "Intact Female", 
#                "breed" : {"$in" : ["NewFoundland", "Labrador Retriever Mix", "Chesapeake Bay Retriever"]}})))
#    elif (int(bt2) > int(bt1) and int(bt2) > int(bt3)):
#        df = pd.DataFrame(list(shelter.read({"age_upon_outcome_in_weeks" : {"$gt":26, "$lt":156}, 
#                "sex_upon_outcome" : "Intact Male", 
#                "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
#                                    " Siberian Husky", "Rottweiler"]}})))
#    elif (int(bt3) > int(bt1) and int(bt3) > int(bt2)):
#        df = pd.DataFrame(list(shelter.read({"age_upon_outcome_in_weeks" : {"$gt":20, "$lt":300}, 
#                "sex_upon_outcome" : "Intact Male", 
#                "breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
#                                    " Bloodhound", "Rottweiler"]}})))
#          
#    return df.to_dict('records')


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    dfp = pd.DataFrame.from_dict(viewData)
    return [
       dcc.Graph(            
           figure = px.pie(dfp, names='breed')
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

#not working with the Mountain/Wildernew or Idividual tracking filters for some reason
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, 
        children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[0,12],dff.iloc[0,13]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1(dff.iloc[0,15]),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]

app